# wprowadzenie i GIL

Upraszcza implementację interpretera kosztem jednowątkowości w jego obrębie.


## Wątki

pakiet threading - do programowania wielowątkowego


Uruchomienie funkcji w nowym wątku jest proste:

In [ ]:
from random import randint
from time import sleep
from threading import Thread

# Thread(group=None, target=None, name=None, args=(), kwargs={})
# group - zarezerwowane na przyszłość
# target - funkcja do wywołania
# name - unikalna nazwa wątku
# args, kwargs - argumenty przekazywane do funkcji


def f(*args, **kwargs):
    print "f({} ,{})".format(args, kwargs)
    sleep(randint(0, 3))
    return


for i in range(5):
    t = Thread(target=f, args=(i,), kwargs={'kwa': 'rg'})
    t.start()  # uruchomienie wątku 
    #t.join()  # oczekiwanie przez wątek wywołujący na wykonanie wątku wywoływanego
    
print "asdf"

Wątek można stworzyć dziedzicząc po klasie Thread

In [ ]:
from threading import Thread


class NewThread(Thread):
    def __init__(self, *args, **kwargs):
        super(NewThread, self).__init__()
        self.args = args
        self.kwargs = kwargs
        
    def run(self):
        # metoda wywoływana przy wywołaniu .start()
        print 'thread is running'
        print self.args
        print self.kwargs
    
    
t = NewThread(1, 2, 3, 4, 5, a=1, b=3)
t.start()
t.join()

## Współdzielenie zasobów

ograniczenie dostępu do seksji krytycznej przez pechanizm:
* Lock/RLock
* Semafor
* Event

### Lock/RLock

Lock i RLock działają podobnie - jedyna różnica to ta, że RLock może być zwolniony tylko przez wątek, który go zajął

In [ ]:
from random import randint
from time import sleep
from threading import Thread, Lock

def print_thread(thread):
    print 'Wykonuje watek {}'.format(thread)
    sleep(randint(0, 2))  # sekcja krytyczna
    print 'Konczy watek {}'.format(thread)
    
    
t1 = Thread(target=print_thread, args=(1,))
t2 = Thread(target=print_thread, args=(2,))
t3 = Thread(target=print_thread, args=(3,))

t1.start()
t2.start()
t3.start()

t1.join()
t2.join()
t3.join()

print 50 * '='
def print_thread_lock(thread, lock):
    #try:  # eksperyment - zwolanianie locka przez inny wątek - na klasie Lock DZIAŁA
    #    lock.release()
    #except:
    #    pass
    lock.acquire()  # lock, zostaje przechwycony
    print 'Wykonuje watek {}'.format(thread)
    sleep(randint(0, 2))  # sekcja krytyczna
    print 'Konczy watek {}'.format(thread)
    lock.release()  # lock zostaje zwolniony - nie zwolnienie blokady zablokuje inne wątki
    

lock = Lock()    
t1 = Thread(target=print_thread_lock, args=(1, lock))
t2 = Thread(target=print_thread_lock, args=(2, lock))
t3 = Thread(target=print_thread_lock, args=(3, lock))

t1.start()
t2.start()
t3.start()

t1.join()
t2.join()
t3.join()

print 50 * '=' + '\n\n'
def print_thread_lock_with(thread, lock):
    with lock:  # to samo co wcześniej tylko z managerem kontekstu
        print 'Wykonuje watek {}'.format(thread)
        sleep(randint(0, 2))  # sekcja krytyczna
        print 'Konczy watek {}'.format(thread)
    

lock = Lock()    
t1 = Thread(target=print_thread_lock_with, args=(1, lock))
t2 = Thread(target=print_thread_lock_with, args=(2, lock))
t3 = Thread(target=print_thread_lock_with, args=(3, lock))

t1.start()
t2.start()
t3.start()

t1.join()
t2.join()
t3.join()

In [ ]:
# RLock

from random import randint
from time import sleep
from threading import Thread, RLock


def print_thread_lock(thread, lock):
    #try:  # eksperyment - zwolanianie locka przez inny wątek - na RLock NIE DZIAŁA
    #    lock.release()
    #except:
    #    pass
    lock.acquire()  # lock, zostaje przechwycony
    print 'Wykonuje watek {}'.format(thread)
    sleep(randint(0, 2))  # sekcja krytyczna
    print 'Konczy watek {}'.format(thread)
    lock.release()  # lock zostaje zwolniony - nie zwolnienie blokady zablokuje inne wątki
    

lock = RLock()    
t1 = Thread(target=print_thread_lock, args=(1, lock))
t2 = Thread(target=print_thread_lock, args=(2, lock))
t3 = Thread(target=print_thread_lock, args=(3, lock))

t1.start()
t2.start()
t3.start()

t1.join()
t2.join()
t3.join()

print 50 * '=' + '\n\n'
def print_thread_lock_with(thread, lock):
    with lock:  # to samo co wcześniej tylko z managerem kontekstu
        print 'Wykonuje watek {}'.format(thread)
        sleep(randint(0, 2))  # sekcja krytyczna
        print 'Konczy watek {}'.format(thread)
    

lock = RLock()    
t1 = Thread(target=print_thread_lock_with, args=(1, lock))
t2 = Thread(target=print_thread_lock_with, args=(2, lock))
t3 = Thread(target=print_thread_lock_with, args=(3, lock))

t1.start()
t2.start()
t3.start()

t1.join()
t2.join()
t3.join()

### Semafor

Semafor pozwala kilku wątkom na dostęp do jednego zasobu - zawiera wewnętrzna wartosć określającą ile wątków może mieć dostęp do zasobu.

Zajmowanie semafora:
- jeżeli wartość semafora jest > 0, wątek obniża wartosć o 1 i kontynuuje wykonanie swojego kodu
- jeżeli wartosć to 0 to wątek obniża wartosć semafora i czeka, aż wartość semafora będzie nieujemna
- jeżeli wartosć semafora < 0 wątek czeka aż ktoś zwolni dostęp do zasobu


In [27]:
from threading import Semaphore, BoundedSemaphore

semaphore = Semaphore(0)
print semaphore._Semaphore__value

# release zwiększa wartość wewnętrzego licznika semafora o 1
semaphore.release()
semaphore.release()
semaphore.release()
# acquire zmniejsza wewnętrzny licznik semafora o 1
semaphore.acquire()
print semaphore._Semaphore__value


print 50 * '='
# BoundedSemaphore nie pozwala na zwiększenie wewnętrzego licznika powyżej początkowego
semaphore = BoundedSemaphore(4)
print semaphore._Semaphore__value
semaphore.acquire()
print semaphore._Semaphore__value
semaphore.release()
print semaphore._Semaphore__value
semaphore.release()


# Semafory mogą byc używane jako managery kontekstu

0
2
4
3
4


ValueError: Semaphore released too many times

mutex to semafor zainicjalizowany wartością 1

### Synchronizacja warunkami (Conditions)

Condition wykrywa zmianę stanu aplikacji i powiadamia inny wątek czekajacy na tę zmianę stanu

In [28]:
from time import sleep
from threading import Thread, Condition

def consumer(condition):
    print 'wewnatrz consumera; '
    condition.acquire()  # acquire i release są konieczne do powiadamiania i czekania na powiadomienie
    print 'consumer oczekuje na warunek; '
    condition.wait()  # bez wywołania wait() consumer kontynuuje działanie
    print 'consumer kontynuuje; '
    condition.release()
    print 'consumerer konczy; '
    

def producer(condition):
    print 'wewnatrz producera; '
    sleep(1)
    condition.acquire()
    sleep(1)
    print 'producer powiadiamia; '
    condition.notify()
    print 'producer zwalnia condition; '
    sleep(3)  
    condition.release()  # póki producer nie zwolni warunku, consumer nie będzie mógł kontynuować
    print 'producer konczy; '
    
    
cond = Condition()
t1 = Thread(target=consumer, args=(cond,))
t2 = Thread(target=producer, args=(cond,))

t1.start()
t2.start()

t1.join()
t2.join()

# Warunki mogą być też używane jako managery kontekstu

wewnatrz producera; wewnatrz consumera; 

consumer oczekuje na warunek; 
producer powiadiamia; 
producer zwalnia condition; 
producer konczy; 
consumer kontynuuje; 
consumerer konczy; 


### Synchronizacja zdarzeniem (Event

Działa podobnie do Condition, z tym że ustawia wewnatrzna flasę na true/false oznaczającą czy inny wątek może kontynuować

In [26]:
from time import sleep
from threading import Thread, Event

# nie przechwytuje się i zwalniania eventu (acquire/release)

def consumer(event):
    print 'wewnatrz consumera; '
    print 'consumer oczekuje na zdarzenie; '
    event.wait()  # oczekuje na flagę eventu będącą true
    print 'consumer kontynuuje; '
    event.clear()
    print 'consumerer konczy; '
    

def producer(event):
    print 'wewnatrz producera; '
    sleep(1)
    print 'producer ustawia flage; '
    event.set()  # inny wątek oczekujacy wykona się od razu
    print 'producer ustawil flage; '
    sleep(3)  
    print 'producer konczy; '
    
    
evt = Event()
t1 = Thread(target=consumer, args=(evt,))
t2 = Thread(target=producer, args=(evt,))

t1.start()
t2.start()

t1.join()
t2.join()

wewnatrz producera; wewnatrz consumera; 

consumer oczekuje na zdarzenie; 
producer ustawia flage; consumer kontynuuje; 

producer ustawil flage; consumerer konczy; 

producer konczy; 


### Komunikacja przy użyciu kolejki (queue)

najważńiejsze 4 metody:
* put - dodaje element do kolejki
* get - pobiera element z kolejki
* task_done - oznacza zakończenie przetwarzania elementu (musi być wywołane)
* join - oczekuje na opróżnienie kolejki

In [35]:
from Queue import Queue  # moduł to queue w pythonie3
from random import randint
from time import sleep
from threading import Thread


def producer(name, queue):
    x = randint(1, 14)
    print 'producer {} puts {}; '.format(name, x)
    queue.put(x)  # jezeli kolejka jest pełna to wyrzuca wyjatek - chyba że dodamy block=True
    print 'producer {} finishes; '.format(name)
    
    
def consumer(name, queue):
    print 'consumer {} starts; '.format(name)
    x = queue.get()  # jeżeli kolejka jest pusta to czeka aż się coś pojawi
    sleep(randint(0, 2))
    print 'consumer {} got {}; '.format(name, x)
    queue.task_done()
    print 'consumer {} finishes; '.format(name)
    

queue = Queue()
producers = [Thread(target=producer, args=(i, queue)) for i in range(5)]
consumers = [Thread(target=consumer, args=(i, queue)) for i in range(5)]

for producer, consumer in zip(producers, consumers):
    producer.start()
    consumer.start()
    

producer 0 puts 5;  producer 1 puts 5; consumer 1 starts; consumer 0 starts; 
 consumer 2 starts;   


producer 1 finishes; producer 2 puts 5; 
producer 3 puts 3; consumer 3 starts; producer 4 puts 8; consumer 4 starts; producer 0 finishes; 





producer 4 finishes; 
 producer 2 finishes; consumer 4 got 8; 

producer 3 finishes; 

consumer 4 finishes; 
consumer 0 got 5; 
consumer 0 finishes; 
consumer 3 got 3; 
consumer 3 finishes; 
consumer 1 got 5; 
consumer 1 finishes; 
consumer 2 got 5; 
consumer 2 finishes; 


## Procesy